<h1>Web Scraping From website Covid Sul-Sel</h1>
sumber : <a href="https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059">Web Scraping HTML Tables with Python</a>

In [1]:
#load library
import numpy as np
import pandas as pd
import lxml.html as lh
from bs4 import BeautifulSoup
import requests

In [2]:
#request url target
URL = 'https://covid19.sulselprov.go.id/data'
#if not succes (200) do this
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36 Edg/89.0.774.50"}
page = requests.get(URL, headers=headers)
print(page)

<Response [200]>


In [3]:
if page.status_code == 200:
    print('Success!')
elif page.status_code == 404:
    print('Not Found.')

Success!


In [4]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [18]:
#tr_elements #see through inspect element in the website

In [6]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[6, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]

In [7]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[2]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"1"
2:"Kota Makassar"
3:"672"
4:"3728"
5:"4400"
6:"0"
7:"12"
8:"147"
9:"159"
10:"157"
11:"29641"
12:"539"
13:"30337"
14:"34880"


In [19]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(2,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 14, the //tr data is not from our table 
    if len(T)!=14:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [20]:
[len(C) for (title,C) in col]

[54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54]

In [21]:
Dict={title:column for (title,column) in col}
data_covid_sulsel=pd.DataFrame(Dict)

In [22]:
data_covid_sulsel.head()

,1,Kota Makassar,672,3728,4400,0,12,147,159,157,29641,539,30337,34880
0,1,Kota Makassar,672,3728,4400,0,12,147,159,157,29641,539,30337,34880
1,2,Kabupaten Gowa,46,823,869,0,1,20,21,38,5324,64,5426,6315
2,3,Kabupaten Luwu Timur,2,525,527,0,0,6,6,5,2710,9,2724,3257
3,4,Kabupaten Sinjai,11,289,300,0,0,1,1,6,2669,14,2689,2990
4,5,Kabupaten Maros,15,334,349,0,0,8,8,5,2217,25,2247,2603


In [23]:
colnames = ["No", 'Kabupaten/Kota', "suspek follow up", "suspek discarded", "suspek total", 'probable follow up', "probable sembuh", 'probable meninggal',
            'probable total', 'konfirmasi dirawat', 'konfirmasi sembuh', 'konfirmasi meninggal', 'konfirmasi total', 'akumulasi']

mengganti nama-nama column https://stackoverflow.com/questions/38101009/changing-multiple-column-names-but-not-all-of-them-pandas-python

In [24]:
#giving new names to the columns
column_indices = [np.arange(0,14)]
new_names = ["No", 'Kabupaten/Kota', "suspek follow up", "suspek discarded", "suspek total", 'probable follow up', "probable sembuh", 'probable meninggal',
            'probable total', 'konfirmasi dirawat', 'konfirmasi sembuh', 'konfirmasi meninggal', 'konfirmasi total', 'akumulasi']
old_names = data_covid_sulsel.columns[column_indices]
data_covid_sulsel.rename(columns=dict(zip(old_names, new_names)), inplace=True)

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:4307: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [25]:
data_covid_sulsel

,No,Kabupaten/Kota,suspek follow up,suspek discarded,suspek total,probable follow up,probable sembuh,probable meninggal,probable total,konfirmasi dirawat,konfirmasi sembuh,konfirmasi meninggal,konfirmasi total,akumulasi
0,1,Kota Makassar,672,3728,4400,0,12,147,159,157,29641,539,30337,34880
1,2,Kabupaten Gowa,46,823,869,0,1,20,21,38,5324,64,5426,6315
2,3,Kabupaten Luwu Timur,2,525,527,0,0,6,6,5,2710,9,2724,3257
3,4,Kabupaten Sinjai,11,289,300,0,0,1,1,6,2669,14,2689,2990
4,5,Kabupaten Maros,15,334,349,0,0,8,8,5,2217,25,2247,2603
5,6,Kabupaten Jeneponto,5,561,566,0,0,2,2,2,1408,11,1421,1989
6,7,Kota Palopo,9,183,192,0,0,1,1,7,1445,21,1473,1666
7,8,Kabupaten Bone,0,340,340,0,0,4,4,5,1296,11,1312,1656
8,9,Kabupaten Luwu Utara,0,318,318,0,0,0,0,1,1300,22,1323,1641
9,10,Kabupaten Pinrang,10,591,601,0,0,5,5,4,928,44,976,1578


In [15]:
#save file in csv format
data_covid_sulsel.to_csv('data/7 mei 2021.csv', index=False)

In [16]:
#get the information
data_covid_sulsel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   No                    27 non-null     object
 1   Kabupaten/Kota        27 non-null     object
 2   suspek follow up      27 non-null     int64 
 3   suspek discarded      27 non-null     int64 
 4   suspek total          27 non-null     int64 
 5   probable follow up    27 non-null     int64 
 6   probable sembuh       27 non-null     int64 
 7   probable meninggal    27 non-null     int64 
 8   probable total        27 non-null     int64 
 9   konfirmasi dirawat    27 non-null     int64 
 10  konfirmasi sembuh     27 non-null     int64 
 11  konfirmasi meninggal  27 non-null     int64 
 12  konfirmasi total      27 non-null     int64 
 13  akumulasi             27 non-null     int64 
dtypes: int64(12), object(2)
memory usage: 3.1+ KB
